In [3]:
## note this forcing data is for the entire CONUS so i need to mask out what isn't in the CRB 
## note: for masking, ufunc functions can be used directly on a DataArray

################################# INPUTS #############################
lat = 48.03125
lon = -121.28125
######################################################################
import numpy as np
import xray
import os
import math 
import gc 
basin = "cascades"
model = "CCSM4"
scenario = "historical"

direc = '/raid/gergel/%s' % "tmin"
tmin_file = "%s_%s_%s.nc" % (model,scenario,"tasmin")
tmin_full = xray.open_dataset(os.path.join(direc,tmin_file))

direc = '/raid/gergel/%s' % "tmax"
tmax_file = "%s_%s_%s.nc" % (model,scenario,"tasmax")
tmax_full = xray.open_dataset(os.path.join(direc,tmax_file))

direc = '/raid/gergel/%s' % "rh"
sh_file = "%s_%s_%s.nc" % (model,scenario,"huss")
q_full = xray.open_dataset(os.path.join(direc,sh_file))

## adjust lat/lon dimensions since the index names are different
tmin_lons_new = tmin_full['lon'].values[tmin_full['lon'].values > 180] - 360 
tmin_full['lon'] = tmin_lons_new
tmax_full['lon'] = tmin_lons_new
q_full['lon'] = tmin_lons_new 

## select out gridcell 
lon_ind = np.where(np.asarray(tmin_full.lon).astype('|S10') == np.str(lon))[0][0]
lat_ind = np.where(np.asarray(tmin_full.lat).astype('|S8') == np.str(lat))[0][0]

# tmin = tmin_full[:,lat_ind,lon_ind]
tmin = tmin_full.isel_points(lon=[lon_ind],lat=[lat_ind])
tmax = tmax_full.isel_points(lon=[lon_ind],lat=[lat_ind])
q = q_full.isel_points(lon=[lon_ind],lat=[lat_ind])

del tmin_full,tmax_full, q_full 
gc.collect() 

600

In [ ]:
## relative humidity 
RH = estimate_relative_humidity(q,t_avg,lat,lon)

## estimate vapor pressures
satvpd = np.exp(1.81 + (((t_avg * 17.27) - 4717.31)/(t_avg - 35.86)))
satvpx = np.exp(1.81 + (((arr_tmin.air_temp_min * 17.27) - 
                         4717.31)/(arr_tmin.air_temp_min - 35.86)))
satvpn = np.exp(1.81 + (((arr_tmax.air_temp_max * 17.27) - 
                         4717.31)/(arr_tmax.air_temp_max - 35.86)))
ambvp = (RH * satvpd) * (1.0/100.0)
RHmax = 100.0 * (ambvp/satvpn)
RHmin = 100.0 * (ambvp/satvpx)

pptdur = np.round((pptamt/wetrat) + 0.49)

## estimate EMCs
EMCmin = estimate_emc(RHmin,arr_tmin)
EMC = estimate_emc(RH,t_avg)
EMCmax = estimate_emc(RHmax,arr_tmax)

phi = lat*0.01745 
decl = 0.41008*np.sin((jdate-82)*0.01745)
daylit = 24*(1-np.arccos(np.tan(phi)*np.tan(decl)))/math.pi

EMCbar = ((daylit * EMCmin + 24 - daylit) * EMCmax)/24.0 

bndryh = ((24.0 - pptdur) * (EMCbar + pptdur) * (0.5 * pptdur + 41.0 ))/24.0 

## to initialize YMC100 at the beginning of a period
YMC100_0 = 5.0 + (5.0 * climat)

MC100 = YMC100 + (bndryh - YMC100) * (1.0 - 0.87*np.exp(-0.24)) ## ymc100 is the mc100 value of the previous day 


